## What is Flow-Spec ? Why do we need to discuss it  here ? 
-----------------------------------------------------------


- Service Class is an idea where a garunteed service within a combination of a controlled load. 
- Use a Queing Mechanism so as to differentiate traffic and Admission control to limit total amount of controlled traffic on a link.
- Any Service can be categorised into specific requirement like how controlled it can be and how garunteed it is withe delay requirements and type of traffic,all this can be collectively called as **Flow-Spec**. 

Flow Spec can be clasisfied further down into two Components - **Tspec and Rspec** 

- Tspec is traffic flow charecterstics. 
- Rspec is more about the state of the network.

Flow in general terms can also be described as set of packets which a unique application sends out and have specifications, hence also Flow-Spec. 

2 things can happen, a Flow can be accepted or turned down by a stream of Routers. 

###When is it turned Down ? 

- Admission Control -> This is where Turn down happens, Imagine you have a 100Mbps line and you have request for 50 flows each with 4Mbps, obviously totaling to 200Mbps, you have to drop down some of the traffic, this is Admission Control 

***Signalling -> This is where you have messages like PATH,RESV,PATH-TEAR,RESV-TEAR in the entire path of routers, we need these messages to convey the signal and acceptance for resource reservation and hence the procotol RSVP - Resource Reservation Protocol.*** 


## This Lab, we will inspect T-spec , Flow-Spec variables, most of the lab is executed via PyEZ, Juniper Framework for Interacting with Devices. 

In [1]:
# Start all docker containers 
import os
import time 
list_of_containers = ['vmx1','vmx2','vmx3','vmx4','vr-xcon']
start_container = [os.system(f'sudo docker start {device}') for device in list_of_containers]

#lets allow container to start, approximately 60seconds-120seconds 
time.sleep(120)
print('Sleeping for 120 Seconds')

vmx1
vmx2
vmx3
vmx4
vr-xcon
Sleeping for 120 Seconds


In [2]:
# Script to initiate packet-captures on Juniper Routers 

from jnpr.junos import Device
from jnpr.junos.utils.start_shell import StartShell
from concurrent.futures import ThreadPoolExecutor
import time 

generated_file_list = [] #this is used to capture all the files generate for auto-fetching 
current_time = time.ctime().split()[3]

def initiate_pktcap(device_intf_tuple): #we have to use tuple here as multi-threading output is tuple 
    host = device_intf_tuple[0]
    interface = device_intf_tuple[1]
    try:
        dev = Device(host=host, user='lab',password='lab123').open()
    except:
        print('Exception in Connnecting')
    with StartShell(dev) as ss:
        interface_renamed = 'ge_'+''.join(interface[3:].split('/'))
        command_string = f'cli -c "monitor traffic interface {interface} write-file /var/tmp/{host}_{interface_renamed}_{current_time}.pcap extensive layer2-headers no-resolve"'
        print(command_string)
        generated_file_list.append(f'{host}_{interface_renamed}_{current_time}.pcap')
        ss.run(command_string,this=None,timeout=20)
        

In [3]:
# At this step, Manually initiate LSP's or clear one of the LSP's to re-trigger flow and RSVP LSP setup

In [5]:
device_interface_mapping = {'vmx1':['ge-0/0/0'], 'vmx2':['ge-0/0/0', 'ge-0/0/1'], 'vmx3':['ge-0/0/1','ge-0/0/2'],
                           'vmx4':['ge-0/0/2']}

#https://learn.acloud.guru/course/list-comprehension-in-python/learn/3baa87dd-d4f6-42a1-9a5d-f29052395e34/28bda287-a243-47f5-b259-0836e677b168/watch
#https://superfastpython.com/threadpoolexecutor-map-vs-submit/#:~:text=Products-,map()%20vs.,With%20the%20ThreadPoolExecutor%20in%20Python&text=Use%20map()%20when%20converting,using%20the%20ThreadPoolExecutor%20in%20Python.

print([(device,interface) for device,interfaces in device_interface_mapping.items() for interface in interfaces])
with ThreadPoolExecutor(max_workers=50) as e:
    e.map(initiate_pktcap, [(device,interface) for device,interfaces in device_interface_mapping.items() for interface in interfaces])
print(generated_file_list)


#ge-0/0/0 in naming wont work, so this has to be split into g0/0/0

    

[('vmx1', 'ge-0/0/0'), ('vmx2', 'ge-0/0/0'), ('vmx2', 'ge-0/0/1'), ('vmx3', 'ge-0/0/1'), ('vmx3', 'ge-0/0/2'), ('vmx4', 'ge-0/0/2')]
cli -c "monitor traffic interface ge-0/0/0 write-file /var/tmp/vmx1_ge_000_23:42:49.pcap extensive layer2-headers no-resolve"
cli -c "monitor traffic interface ge-0/0/2 write-file /var/tmp/vmx4_ge_002_23:42:49.pcap extensive layer2-headers no-resolve"
cli -c "monitor traffic interface ge-0/0/1 write-file /var/tmp/vmx2_ge_001_23:42:49.pcap extensive layer2-headers no-resolve"
cli -c "monitor traffic interface ge-0/0/0 write-file /var/tmp/vmx2_ge_000_23:42:49.pcap extensive layer2-headers no-resolve"
cli -c "monitor traffic interface ge-0/0/1 write-file /var/tmp/vmx3_ge_001_23:42:49.pcap extensive layer2-headers no-resolve"
cli -c "monitor traffic interface ge-0/0/2 write-file /var/tmp/vmx3_ge_002_23:42:49.pcap extensive layer2-headers no-resolve"
['vmx1_ge_000_23:42:49.pcap', 'vmx4_ge_002_23:42:49.pcap', 'vmx2_ge_000_23:42:49.pcap', 'vmx2_ge_001_23:42:49.p

In [6]:
#function to open t-shark for specified files 
#first copy files to local-host 
from jnpr.junos import Device as dev 
from jnpr.junos.utils.scp import SCP
from concurrent.futures import ThreadPoolExecutor


import os 
import sys 
import logging 

def file_to_fetch(file_to_get):
    host = file_to_get.split('_')[0]
    print(f'These will be fetched from {host} -> {file_to_get} to /var/tmp')
    d = dev(user='lab',password='lab123', host=host).open()
    with SCP(d) as scp:
        if host in file_to_get: # This is to filter-out hosts - wrongfile mapping
            scp_file = scp.get(f'/var/tmp/{file_to_get}', local_path=f'/var/tmp/{file_to_get}')
            

with ThreadPoolExecutor(max_workers=50) as e:
    e.map(file_to_fetch, [files for files in generated_file_list])
    


These will be fetched from vmx1 -> vmx1_ge_000_23:42:49.pcap to /var/tmp
These will be fetched from vmx4 -> vmx4_ge_002_23:42:49.pcap to /var/tmp
These will be fetched from vmx2 -> vmx2_ge_000_23:42:49.pcap to /var/tmp
These will be fetched from vmx2 -> vmx2_ge_001_23:42:49.pcap to /var/tmp
These will be fetched from vmx3 -> vmx3_ge_001_23:42:49.pcap to /var/tmp
These will be fetched from vmx3 -> vmx3_ge_002_23:42:49.pcap to /var/tmp
These will be fetched from vmx1 -> vmx1_ge_000_23:42:49.pcap to /var/tmp
These will be fetched from vmx4 -> vmx4_ge_002_23:42:49.pcap to /var/tmp
These will be fetched from vmx2 -> vmx2_ge_001_23:42:49.pcap to /var/tmp
These will be fetched from vmx2 -> vmx2_ge_000_23:42:49.pcap to /var/tmp
These will be fetched from vmx3 -> vmx3_ge_001_23:42:49.pcap to /var/tmp
These will be fetched from vmx3 -> vmx3_ge_002_23:42:49.pcap to /var/tmp


In [7]:
!tshark -r /var/tmp/vmx1_ge_000_23:42:49.pcap #vmx1 

    1   0.000000     9.9.12.1 → 224.0.0.5    OSPF 116 Hello Packet
    2   0.000558     9.9.12.1 → 224.0.0.5    OSPF 116 Hello Packet
    3   0.002061     9.9.12.1 → 9.9.12.2     RSVP 264 Component Messages Dissected
    4   0.069379     9.9.12.2 → 9.9.12.1     RSVP 66 ACK Message. 
    5   0.101826     9.9.12.2 → 9.9.12.1     RSVP 230 RESV Message. SESSION: IPv4-LSP, Destination 4.4.4.4, Short Call ID 0, Tunnel ID 22821, Ext ID 1010101. FILTERSPEC: IPv4-LSP, Tunnel Source: 1.1.1.1, Short Call ID: 0, LSP ID: 1. 
    6   0.102198     9.9.12.1 → 9.9.12.2     RSVP 76 ACK Message. 
    7   0.594386     9.9.12.1 → 1.1.1.1      UDP 100 8503 → 8503 Len=32
    8   0.596932     9.9.12.1 → 1.1.1.1      UDP 86 8503 → 8503 Len=32
    9   3.017541     9.9.12.1 → 9.9.12.2     RSVP 96 HELLO Message. 
   10   3.076198     9.9.12.2 → 9.9.12.1     RSVP 86 HELLO Message. 
   11   4.557787     9.9.12.2 → 224.0.0.5    OSPF 106 Hello Packet
   12   9.217569     9.9.12.1 → 224.0.0.5    OSPF 116 Hello Packet


In [8]:
!tshark -r /var/tmp/vmx1_ge_000_23:42:49.pcap -Y "frame.number==3" -V

Frame 3: 264 bytes on wire (2112 bits), 264 bytes captured (2112 bits)
    Encapsulation type: Juniper Ethernet (83)
    Arrival Time: Aug 21, 2022 23:44:26.552271000 UTC
    [Time shift for this packet: 0.000000000 seconds]
    Epoch Time: 1661125466.552271000 seconds
    [Time delta from previous captured frame: 0.001503000 seconds]
    [Time delta from previous displayed frame: 0.000000000 seconds]
    [Time since reference or first frame: 0.002061000 seconds]
    Frame Number: 3
    Frame Length: 264 bytes (2112 bits)
    Capture Length: 264 bytes (2112 bits)
    [Frame is marked: False]
    [Frame is ignored: False]
    [Protocols in frame: juniper:eth:ethertype:ip:rsvp]
Juniper Ethernet
    Magic Number: 0x4d4743
    .... ...0 = Direction: Out (0x0)
    .... ..0. = L2 header presence: Present (0x0)
    Extension(s) Total length: 16
        Device Media Type Extension TLV #3, length: 1
            Device Media Type: Ethernet (1)
        Logical Interface Encapsulation Extension TL

In [9]:
!tshark -r /var/tmp/vmx2_ge_000_23:42:49.pcap #vmx2

    1   0.000000     9.9.12.1 → 224.0.0.5    OSPF 106 Hello Packet
    2   0.000005     9.9.12.1 → 224.0.0.5    OSPF 106 Hello Packet
    3   0.032952     9.9.12.1 → 9.9.12.2     RSVP 254 Component Messages Dissected
    4   0.033601     9.9.12.2 → 9.9.12.1     RSVP 76 ACK Message. 
    5   0.094203     9.9.12.2 → 9.9.12.1     RSVP 240 RESV Message. SESSION: IPv4-LSP, Destination 4.4.4.4, Short Call ID 0, Tunnel ID 22821, Ext ID 1010101. FILTERSPEC: IPv4-LSP, Tunnel Source: 1.1.1.1, Short Call ID: 0, LSP ID: 1. 
    6   0.117697     9.9.12.1 → 9.9.12.2     RSVP 66 ACK Message. 
    7   3.039734     9.9.12.1 → 9.9.12.2     RSVP 86 HELLO Message. 
    8   3.039813     9.9.12.2 → 9.9.12.1     RSVP 96 HELLO Message. 
    9   4.554594     9.9.12.2 → 224.0.0.5    OSPF 116 Hello Packet
   10   9.231706     9.9.12.1 → 224.0.0.5    OSPF 106 Hello Packet
   11  12.046804     9.9.12.1 → 9.9.12.2     RSVP 86 HELLO Message. 
   12  12.046887     9.9.12.2 → 9.9.12.1     RSVP 96 HELLO Message. 
   13

In [10]:
!tshark -r /var/tmp/vmx2_ge_000_23:42:49.pcap -Y "frame.number==5" -V

Frame 5: 240 bytes on wire (1920 bits), 240 bytes captured (1920 bits)
    Encapsulation type: Juniper Ethernet (83)
    Arrival Time: Aug 21, 2022 23:44:26.626497000 UTC
    [Time shift for this packet: 0.000000000 seconds]
    Epoch Time: 1661125466.626497000 seconds
    [Time delta from previous captured frame: 0.060602000 seconds]
    [Time delta from previous displayed frame: 0.000000000 seconds]
    [Time since reference or first frame: 0.094203000 seconds]
    Frame Number: 5
    Frame Length: 240 bytes (1920 bits)
    Capture Length: 240 bytes (1920 bits)
    [Frame is marked: False]
    [Frame is ignored: False]
    [Protocols in frame: juniper:eth:ethertype:ip:rsvp]
Juniper Ethernet
    Magic Number: 0x4d4743
    .... ...0 = Direction: Out (0x0)
    .... ..0. = L2 header presence: Present (0x0)
    Extension(s) Total length: 16
        Device Media Type Extension TLV #3, length: 1
            Device Media Type: Ethernet (1)
        Logical Interface Encapsulation Extension TL